# Etiquetado IoT-23

In [14]:
%pip install parsezeeklogs 
%pip install elasticsearch

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [16]:
from parsezeeklogs import ParseZeekLogs
from elasticsearch import Elasticsearch

In [21]:
import csv
import json
import sys
import time
import os
from parsezeeklogs import ParseZeekLogs
from datetime import datetime

In [19]:
# Define the conn.log and labels.csv file paths
connlog_path = "conn.log"
labelsfile_path = "labels.csv"
helptext = "Usage: python3 flaber.py conn.log labels.csv"
logfile = "flaber.log"


In [20]:
def labeler(log_record, labelsjson):
    # Parsing flows to json
    log_record = json.loads(log_record)
    labelsjson = json.loads(labelsjson)
    
    if len(labelsjson) == 0:
        return False
    
    if len(log_record) == 0:
        return False
    
    i=1
    ok_labels=[]
    # For every label in label file
    for row in labelsjson:        
        try: 
            if len(row.keys()) != 8:
                print("ERROR: labels.csv fie has a wrong format: {}".format(str(row)))
                logic_logfile.write("ERROR: labels.csv fie has a wrong format: {}".format(str(row)))
                pass

            labeled_row = {}
            i+=1
            flowdata = log_record[str(row["Field"])]
            labeled_row["id"] = row["Id"]
            labeled_row["ok"] = False
            # Checking multiple connected label criteria
            if len(row["connector"]) > 1:
                labeled_row["connect"] = row["connector"]
            else:
                labeled_row["connect"] = False
            
            # Checking comparators
            if row["Comparator"] == "eq":

                # Comparing data
                if str(flowdata) == str(row["Data"]):
                    # Adding labels
                    labeled_row["label"] = row["Label"]
                    labeled_row["ok"] = True
                    
            if row["Comparator"] == "gt":
                if int(flowdata) > int(row["Data"]):
                    labeled_row["label"] = row["Label"]
                    labeled_row["ok"] = True
                    
            if row["Comparator"] == "lt":
                if int(flowdata) < int(row["Data"]):
                    labeled_row["label"] = row["Label"]
                    labeled_row["ok"] = True

            # Accounting ok labels
            ok_labels.append(labeled_row)

        except Exception as e:
            print("Exception in labeler function: {}".format(str(e)))
            logic_logfile.write("Exception in labeler function: {} \n ".format(str(e)))
            logic_logfile.write("Exception at row: {} \n ".format(str(row)))
            logic_logfile.write("Exception at flow {} \n ".format(str(log_record)))
           
    # Labeling the ok ones
    try:
        for label in ok_labels:
            haslabel = False    
            if label["ok"]:
                haslabel = True
                # Formating label string
                
                
                if not "label" in log_record:
                    log_record["label"] = ""
                elif(len(log_record["label"]) > 1 and not log_record["label"][-1] == "-"):
                    log_record["label"] += "-"
                
                # Cheking multiple connected label criterias that were ok
                if label["connect"] and label["connect"].split(" ")[1] and label["connect"].split(" ")[1] != " ":
                    conn_id = label["connect"].split(" ")[1]
                    # Combining multiple ok labeled criterias

                    if ok_labels[int(conn_id) - 1]["ok"] and label["ok"]:
                        # Labeling
                    
                        if label["label"] not in log_record["label"]:
                            log_record["label"] += label["label"]
                            
                else:
                    # Labeling
                    if label["label"] not in log_record["label"]:
                        log_record["label"] += label["label"]

    except Exception as e:
        raise(e)

    try:
        if not "label" in log_record or len(log_record["label"]) < 1:
            log_record["label"] = "-"
        else:
            if str(log_record["label"])[-1] == "-":
                log_record["label"] = str(log_record["label"])[0:-1]

    except Exception as e:
       raise(e)

    return log_record


In [22]:
# Writing log file data
logic_logfile = open(logfile,"w")
datetimelog = datetime.now().strftime("%d/%m/%y %H:%M:%S")
logic_logfile.write("Flaber started at:" + str(datetime) + "\n")

try:
    # Open the CSV  
    f = open(labelsfile_path, 'r' )
    # Change each fieldname to the appropriate field name. I know, so difficult.  
    reader = csv.DictReader(f)  
    # Parse the CSV into JSON  
except Exception as e:
    print("Exception opening conn.log file: {}".format(str(e)))
    logic_logfile.write("Exception opening conn.log file: {} \n ".format(str(e)))
        

try:
    result_file = "out.json"
    
    labelsjson = json.dumps( [ row for row in reader ] )  
    conn_file_lines = os.popen('wc -l ' + str(connlog_path) + ' | cut -d" " -f1').read()
    conn_file_lines = int(conn_file_lines) - 8
    info_msg = "Labeling conn.log file: " + str(connlog_path) + " with " + str(conn_file_lines) +  " lines \n"
    logic_logfile.write(info_msg)
    print(info_msg)

    count = 1
    # Analyzing, labeling and writing out.csv file
    with open(result_file,"w") as outfile:
        # Reading flows from conn.log
        for log_record in ParseZeekLogs(connlog_path, output_format="json", safe_headers=False):
            if log_record is not None:
                # Analyzing and labelig flows
                log_record = labeler(log_record, labelsjson)
                #print(json.dumps(log_record))
                # Writing resulting labeled flows
                outfile.write(json.dumps(log_record) + "\n")
                count += 1
                if (count % 100000 == 0):
                    print("Analyzing {} flows ".format(count))
    
    print("Labeled: {} of {} flows".format(count, str(conn_file_lines)))
    print("Labeled flows written to: {}".format(result_file))

except Exception as e:
    print("Exception opening {} file: {}".format(result_file, str(e)))
    logic_logfile.write("Exception opening {} file: {} \n ".format(result_file, str(e)))
    raise(e)

datetimelog = datetime.now().strftime("%d/%m/%y %H:%M:%S")
logic_logfile.write("Flaber finished at:" + str(datetimelog) + "\n")

try:
    f.close()
    logic_logfile.close()
except Exception as e:
    print("Exception closing files: {}".format(str(e)))
    logic_logfile.write("Exception closing files: {} \n ".format(str(e)))

Exception opening conn.log file: [Errno 2] No such file or directory: 'labels.csv'
Exception opening out.json file: name 'reader' is not defined


NameError: name 'reader' is not defined

## CTU-Malware 1

C&C
PartOfAHorizontalPortScan
PartOfAHorizontalPortScan
PartOfAHorizontalPortScan
PartOfAHorizontalPortScan

In [ ]:
# Create the binary-label column
df['binary-label'] = 0  # Initialize with 0
df['label'] = ''
rule_1 = (df['id.resp_p'] == 2407)
rule_2 = (df['id.resp_p'] == 9527)
rule_3 = (df['id.resp_p'] == 8080)
rule_4 = (df['id.resp_p'] == 23)
rule_5 = (df['id.resp_p'] == 2323)

# Create the label column

df.loc[rule_1, 'label'] = 'C&C'
df.loc[rule_2, 'label'] = 'PartOfAHorizontalPortScan'
df.loc[rule_3, 'label'] = 'PartOfAHorizontalPortScan'
df.loc[rule_4, 'label'] = 'PartOfAHorizontalPortScan'
df.loc[rule_5, 'label'] = 'PartOfAHorizontalPortScan'

## CTU Malware 17 revisar

In [ ]:
# Create the binary-label column
df['binary-label'] = 0  # Initialize with 0
df['label'] = ''
rule_1 = (df['id.resp_h'] == "176.32.33.171")
rule_2 = (df['id.resp_p'] == 37215)
rule_3 = (df['id.resp_p'] == 666)
rule_4 = (df["id.resp_p"] == 80) & (df["conn_state"] == "S0")
rule_6 = (df["id.resp_p"] == 8080) & (df["conn_state"] == "S0")
rule_7 = (df["conn_state"] == "S0") & (df["id.resp_p"] == 8081)
rule_8 = (df["id.resp_p"] == 8081) & (df["id.resp_p"] == 52869)
rule_9 = (df["conn_state"] == "S0") & (df['id.resp_h'] == "69.255.24.137")
rule_10 = (df['id.resp_h'] == "69.255.24.137")
rule_11 = (df["id.resp_p"] == 52869)"

# Create the label column
df.loc[rule_1, 'label'] = 'C&C'
df.loc[rule_2, 'label'] = 'Okiru'
df.loc[rule_3, 'label'] = 'HeartBeat'
df.loc[rule_4, 'label'] = 'PartOfAHorizontalPortScan'
df.loc[rule_6, 'label'] = 'C&C'
df.loc[rule_7, 'label'] = 'PartOfAHorizontalPortScan'
df.loc[rule_8, 'label'] = 'PartOfAHorizontalPortScan'
df.loc[rule_9, 'label'] = 'PartOfAHorizontalPortScan'
df.loc[rule_10, 'label'] = 'Attack'
df.loc[rule_11, 'label'] = 'DDoS'

## CTU Malware 20

In [ ]:
df['binary-label'] = 0  # Initialize with 0
df['label'] = '
rule_1 = (df['id.resp_h'] == "66.85.157.90")

In [ ]:
df.loc[rule_1, 'label'] = 'C&C-Torii'

## CTU Malware 21

In [2]:
import pandas as pd
file_path = "/root/bbdd/logs-zeek/iot-23-logs/CTU-IoT-Malware-Capture-21-1/conn_stadistics.log"
df = pd.read_json(file_path, lines=True)

In [4]:
df['binary-label'] = 0  # Initialize with 0
df['label'] = 'benign'
rule_1 = (df['id.resp_h'] == "66.85.157.90")

In [8]:
df.loc[rule_1, ['label', 'binary-label']] = ['C&C-Torii', 1]

In [9]:
filtered_df = df[df['label'] == 'C&C-Torii']

# Print the filtered DataFrame
print(filtered_df)

                ts            startTime                 uid        id.orig_h  \
20    1.538573e+09  2018-10-03 13:25:33  CRy07q1Hf284gJha32  192.168.100.113   
22    1.538573e+09  2018-10-03 13:25:40   CjDSurNqQDaJmk0e3  192.168.100.113   
25    1.538573e+09  2018-10-03 13:25:48  CRTqJt3H1GzHSvdm3b  192.168.100.113   
2034  1.538573e+09  2018-10-03 13:26:56  CoxIFH203hWqO4Ci57  192.168.100.113   
2035  1.538607e+09  2018-10-03 22:52:07  CQiI8R12FyaTZxgVpd  192.168.100.113   
2046  1.538607e+09  2018-10-03 22:53:14   CIrQLFst6wt0R8HQ3  192.168.100.113   
2048  1.538607e+09  2018-10-03 22:53:21  C2qsJB2NUWQqNtjLC4  192.168.100.113   
2049  1.538607e+09  2018-10-03 22:53:29   CcQWlru6OW6KqeAgg  192.168.100.113   
2050  1.538607e+09  2018-10-03 22:53:56  CXeonO3mUm8UXjemR4  192.168.100.113   
2052  1.538607e+09  2018-10-03 22:54:04  CbiPyH1M6zlvf4Fjzc  192.168.100.113   
2053  1.538607e+09  2018-10-03 22:54:12   CMTVcX1fw6ZIPLnpX  192.168.100.113   
2055  1.538607e+09  2018-10-03 22:54:36 

## CTU Malware 3

In [ ]:
df['binary-label'] = 0  # Initialize with 0
df['label'] = ''
rule_1 = (df['id.resp_p'] == 2407)
rule_2 = (df["conn_state"] == "S0") & (df["id.resp_p"] == 59353)
rule_3 = (df["conn_state"] == "S0") & (df["id.resp_p"] == 22)
rule_4 = (df["conn_state"] == "SF") & (df["id.resp_p"] == 22)

df.loc[rule_1, 'label'] = 'C&C'
df.loc[rule_2, 'label'] = 'PartOfAHorizontalPortScan'
df.loc[rule_3, 'label'] = 'PartOfAHorizontalPortScan'
df.loc[rule_4, 'label'] = 'PartOfAHorizontalPortScan'


## CTU Malware 33

In [ ]:
df['binary-label'] = 0  # Initialize with 0
df['label'] = ''
rule_1 = (df['id.resp_h'] == "176.32.33.171")
rule_2 = (df['id.resp_p'] == 37215)
rule_3 = (df['id.resp_p'] == 666)
rule_4 = (df["id.resp_p"] == 80) & (df["conn_state"] == "S0")
rule_5 = (df["conn_state"] == "S0") & (df["id.resp_p"] == 52869)
rule_6 = (df["conn_state"] == "S0") & (df["id.resp_p"] == 8081)

df.loc[rule_1, 'label'] = 'C&C'
df.loc[rule_2, 'label'] = 'Okiru'
df.loc[rule_3, 'label'] = 'HeartBeat'
df.loc[rule_4, 'label'] = 'PartOfAHorizontalPortScan'
df.loc[rule_5, 'label'] = 'PartOfAHorizontalPortScan'
df.loc[rule_6, 'label'] = 'PartOfAHorizontalPortScan'


## CTU Malware 34

df['binary-label'] = 0  # Initialize with 0
df['label'] = ''
rule_1 = (df['id.resp_p'] == "6667")
rule_2 = (df['id.resp_p'] == "63798")
rule_3 = (df['id.resp_p'] == "256")
rule_4 = (df['id.resp_h'] == "123.59.209.185")
rule_5 = (df['id.resp_h'] == "71.61.66.148")
rule_6 = (df['id.resp_h'] == "74.91.117.248")
rule_7 = (df['id.resp_p'] == "5376")


df.loc[rule_1, 'label'] = 'C&C'
df.loc[rule_2, 'label'] = "PartOfAHorizontalPortscan"
df.loc[rule_3, 'label'] = 'PartOfAHorizontalPortscan'
df.loc[rule_4, 'label'] = 'DDoS'
df.loc[rule_5, 'label'] = 'DDoS'
df.loc[rule_6, 'label'] = 'DDoS'
df.loc[rule_7, 'label'] = 'DDoS'

In [19]:
import json

input_file = "/root/bbdd/logs-zeek/iot-23-logs/CTU-IoT-Malware-Capture-34-1/conn_stadistics.log"
output_file = "/root/bbdd/logs-zeek/iot-23-logs/labeled-csv/json-labeled-CTU-IoT-Malware-Capture-34-1.csv"

label_1 = "C&C"
label_2 = "PartOfAHorizontalPortscan"
label_3 = "DDoS"

# Initialize label checks
label_checked = [False] * 3

all_keys = ["ts", "startTime", "uid", "id.orig_h", "id.orig_p", "id.resp_h", "id.resp_p", "proto", "service", "duration", "orig_bytes", 
            "resp_bytes", "conn_state", "local_orig", "local_resp", "missed_bytes", "history", "orig_pkts", "orig_ip_bytes", "resp_pkts", 
            "resp_ip_bytes", "tunnel_parents", "orig_bytes_mean", "resp_bytes_mean", "orig_bytes_std", "resp_bytes_std", "orig_bytes_mean_nocero", 
            "resp_bytes_mean_nocero", "orig_bytes_std_nocero", "resp_bytes_std_nocero", "orig_bytes_min", "resp_bytes_min", "orig_bytes_max", 
            "resp_bytes_max", "orig_pkts_nocero", "resp_pkts_nocero", "orig_pkts_cero", "resp_pkts_cero", "time_mean", "time_std", "time_min", 
            "time_max", "orig_time_mean", "orig_time_std", "orig_time_min", "orig_time_max", "resp_time_mean", "resp_time_std", "resp_time_min", "resp_time_max"]


with open(input_file, "r") as f_in, open(output_file, "w") as f_out:
    f_out.write("ts,startTime,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,orig_bytes_mean,resp_bytes_mean,orig_bytes_std,resp_bytes_std,orig_bytes_mean_nocero,resp_bytes_mean_nocero,orig_bytes_std_nocero,resp_bytes_std_nocero,orig_bytes_min,resp_bytes_min,orig_bytes_max,resp_bytes_max,orig_pkts_nocero,resp_pkts_nocero,orig_pkts_cero,resp_pkts_cero,time_mean,time_std,time_min,time_max,orig_time_mean,orig_time_std,orig_time_min,orig_time_max,resp_time_mean,resp_time_std,resp_time_min,resp_time_max,label,binary-label\n")
    for line in f_in:
        data = json.loads(line)
        for key in all_keys:
            if key not in data:
                data[key] = ""
        binary_label = 0
        labels = []  # Start empty
        if data["id.resp_p"] == "6667" and not label_checked[0]:
            labels.append(label_1)
            binary_label = 1
            label_checked[0] = True
        if data["id.resp_p"] == "63798" and not label_checked[1]:
            labels.append(label_2)
            binary_label = 1
            label_checked[1] = True
        if data["id.resp_p"] == "256" and not label_checked[1]:
            labels.append(label_2)
            binary_label = 1
            label_checked[1] = True
        if data["id.resp_h"] == "123.59.209.185" and not label_checked[2]:
            labels.append(label_3)
            binary_label = 1
            label_checked[2] = True
        if data["id.resp_h"] == "71.61.66.148" and not label_checked[2]:
            labels.append(label_3)
            binary_label = 1
            label_checked[2] = True
        if data["id.resp_h"] == "74.91.117.248" and not label_checked[2]:
            labels.append(label_3)
            binary_label = 1
            label_checked[2] = True
        if data["id.resp_p"] == "5376" and not label_checked[2]:
            labels.append(label_3)
            binary_label = 1
            label_checked[2] = True
        

        # Reset label checks if all labels are checked
        # If no labels are added, assign "benign"
        if not labels:
            labels.append("benign")
        
        values = [str(data[key]) for key in all_keys]
        labels_joined = '-'.join(labels)
        csv_line = ','.join(values) + f',{labels_joined},{binary_label}\n'        
        f_out.write(csv_line)

        label_checked = [False] * 3

## CTU Malware 35

df['binary-label'] = 0  # Initialize with 0
df['label'] = ''
rule_1 = (df['id.resp_h'] == "104.248.160.24")
rule_2 = (df['id.resp_h'] == "104.248.160.24") & (df['resp_ip_bytes']>30000)
rule_3 = (df['id.resp_h'] == "110.183.76.177")
rule_4 = (df['id.resp_h'] == "112.27.30.87")
rule_5 = (df['id.resp_h'] == "85.217.225.181")
rule_6 = (df['id.resp_p'] == "992")
rule_7 = (df['id.resp_h'] == "209.97.190.136")
rule_8 = (df['id.resp_h'] == "173.113.172.138")
rule_9 = (df['id.resp_h'] == "216.18.168.16")
rule_10 = (df['id.resp_h'] == "24.165.115.195")
rule_11 = (df['id.resp_h'] == "54.39.87.104")

df.loc[rule_1, 'label'] = 'C&C'
df.loc[rule_2, 'label'] = "FileDownload"
df.loc[rule_3, 'label'] = 'Attack'
df.loc[rule_4, 'label'] = 'Attack'
df.loc[rule_5, 'label'] = 'Attack'
df.loc[rule_6, 'label'] = 'DDoS'
df.loc[rule_7, 'label'] = 'DDoS'
df.loc[rule_8, 'label'] = 'DDoS'
df.loc[rule_9, 'label'] = 'DDoS'
df.loc[rule_10, 'label'] = 'DDoS'
df.loc[rule_11, 'label'] = 'DDoS'

In [20]:
import json

input_file = "/root/bbdd/logs-zeek/iot-23-logs/CTU-IoT-Malware-Capture-35-1/conn_stadistics.log"
output_file = "/root/bbdd/logs-zeek/iot-23-logs/labeled-csv/json-labeled-CTU-IoT-Malware-Capture-35-1.csv"

label_1 = "C&C"
label_2 = "FileDownload"
label_3 = "Attack"
label_4 = "DDoS"

# Initialize label checks
label_checked = [False] * 4

all_keys = ["ts", "startTime", "uid", "id.orig_h", "id.orig_p", "id.resp_h", "id.resp_p", "proto", "service", "duration", "orig_bytes", 
            "resp_bytes", "conn_state", "local_orig", "local_resp", "missed_bytes", "history", "orig_pkts", "orig_ip_bytes", "resp_pkts", 
            "resp_ip_bytes", "tunnel_parents", "orig_bytes_mean", "resp_bytes_mean", "orig_bytes_std", "resp_bytes_std", "orig_bytes_mean_nocero", 
            "resp_bytes_mean_nocero", "orig_bytes_std_nocero", "resp_bytes_std_nocero", "orig_bytes_min", "resp_bytes_min", "orig_bytes_max", 
            "resp_bytes_max", "orig_pkts_nocero", "resp_pkts_nocero", "orig_pkts_cero", "resp_pkts_cero", "time_mean", "time_std", "time_min", 
            "time_max", "orig_time_mean", "orig_time_std", "orig_time_min", "orig_time_max", "resp_time_mean", "resp_time_std", "resp_time_min", "resp_time_max"]

with open(input_file, "r") as f_in, open(output_file, "w") as f_out:
    f_out.write("ts,startTime,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,orig_bytes_mean,resp_bytes_mean,orig_bytes_std,resp_bytes_std,orig_bytes_mean_nocero,resp_bytes_mean_nocero,orig_bytes_std_nocero,resp_bytes_std_nocero,orig_bytes_min,resp_bytes_min,orig_bytes_max,resp_bytes_max,orig_pkts_nocero,resp_pkts_nocero,orig_pkts_cero,resp_pkts_cero,time_mean,time_std,time_min,time_max,orig_time_mean,orig_time_std,orig_time_min,orig_time_max,resp_time_mean,resp_time_std,resp_time_min,resp_time_max,label,binary-label\n")

    for line in f_in:
        data = json.loads(line)
        for key in all_keys:
            if key not in data:
                data[key] = ""
        binary_label = 0
        labels = []  # Start empty
        if data["id.resp_h"] == "104.248.160.24" and not label_checked[0]:
            labels.append(label_1)
            binary_label = 1
            label_checked[0] = True
        if data["id.resp_h"] == "104.248.160.24" and data["resp_ip_bytes"] > 30000 and not label_checked[1]:
            labels.append(label_2)
            binary_label = 1
            label_checked[1] = True
        if data["id.resp_h"] == "110.183.76.177" and not label_checked[2]:
            labels.append(label_3)
            binary_label = 1
            label_checked[2] = True
        if data["id.resp_h"] == "112.27.30.87" and not label_checked[2]:
            labels.append(label_3)
            binary_label = 1
            label_checked[2] = True
        if data["id.resp_h"] == "85.217.225.181" and not label_checked[2]:
            labels.append(label_3)
            binary_label = 1
            label_checked[2] = True
        if data["id.resp_p"] == "992" and not label_checked[3]:
            labels.append(label_4)
            binary_label = 1
            label_checked[3] = True
        if data["id.resp_h"] == "209.97.190.136" and not label_checked[3]:
            labels.append(label_4)
            binary_label = 1
            label_checked[3] = True
        if data["id.resp_h"] == "173.113.172.138" and not label_checked[3]:
            labels.append(label_4)
            binary_label = 1
            label_checked[3] = True
        if data["id.resp_h"] == "216.18.168.16" and not label_checked[3]:
            labels.append(label_4)
            binary_label = 1
            label_checked[3] = True
        if data["id.resp_h"] == "24.165.115.195" and not label_checked[3]:
            labels.append(label_4)
            binary_label = 1
            label_checked[3] = True
        if data["id.resp_h"] == "54.39.87.104" and not label_checked[3]:
            labels.append(label_4)
            binary_label = 1
            label_checked[3] = True

        # Reset label checks if all labels are checked
        # If no labels are added, assign "benign"
        if not labels:
            labels.append("benign")
        
        values = [str(data[key]) for key in all_keys]
        labels_joined = '-'.join(labels)
        csv_line = ','.join(values) + f',{labels_joined},{binary_label}\n'        
        f_out.write(csv_line)

        label_checked = [False] * 4

## CTU Malware 36

In [ ]:
import pandas as pd
file_path = "/root/bbdd/logs-zeek/iot-23-logs/CTU-IoT-Malware-Capture-36-1/conn_stadistics.log"
df = pd.read_json(file_path, lines=True)

df['binary-label'] = 0  # Initialize with 0
df['label'] = 'benign'
rule_1 = (df['id.resp_p'] == "57722")
rule_2 = (df['id.resp_p'] == "37215")
rule_3 = (df['id.resp_h'] == "185.130.215.13")
rule_4 = (df['id.resp_h'] == "103.56.26.82")

#df.loc[rule_1, 'label'] = 'C&C'
#df.loc[rule_2, 'label'] = "Okiru"
#df.loc[rule_3, 'label'] = 'HeartBeat'
#df.loc[rule_4, 'label'] = 'Okiru-Attack'

label_1 = 'C&C'
label_2 = 'Okiru'
label_3 = 'HeartBeat'
label_4 = 'Okiru-Attack'
# Add more labels as needed

# Concatenate the labels with a hyphen if multiple rules are True
combined_label = (label_1 + '-' if rule_1.any() else '') + (label_2 + '-' if rule_2.any() else '') + (label_3 + '-' if rule_3.any() else '') + (label_4 + '-' if rule_4.any() else '') 
# Add more labels and corresponding rules as needed

# Assign the combined label to the 'label' column
df.loc[rule_1 | rule_2 | rule_3 | rule_4, 'label'] = combined_label
df.to_csv('/root/bbdd/logs-zeek/iot-23-logs/labeled-csv/labeled-CTU-IoT-Malware-Capture-36-1.csv', index=False)

In [1]:
import pandas as pd

# Define the file path
file_path = "/root/bbdd/logs-zeek/iot-23-logs/CTU-IoT-Malware-Capture-36-1/conn_stadistics.log"

# Define chunk size
chunk_size = 10000

# Define initial labels and conditions
initial_label = 'benign'
initial_binary_label = 0

# Define the conditions and corresponding labels
conditions_labels = [
    ((lambda row: row['id.resp_p'] == "57722"), 'C&C'),
    ((lambda row: row['id.resp_p'] == "37215"), 'Okiru'),
    ((lambda row: row['id.resp_h'] == "185.130.215.13"), 'HeartBeat'),
    ((lambda row: row['id.resp_h'] == "103.56.26.82"), 'Okiru-Attack')
]

# Process the DataFrame in chunks
chunks = pd.read_json(file_path, lines=True, chunksize=chunk_size)
for chunk in chunks:
    # Initialize labels for the chunk
    chunk['label'] = initial_label
    chunk['binary-label'] = initial_binary_label
    
    # Apply conditions and labels for each chunk
    for condition, label in conditions_labels:
        chunk.loc[condition(chunk), 'label'] = label
        chunk.loc[condition(chunk), 'binary-label'] = 1

    # Save the chunk to a new CSV file
    chunk.to_csv('/root/bbdd/logs-zeek/iot-23-logs/labeled-csv/labeled-CTU-IoT-Malware-Capture-36-1.csv', mode='a', header=False, index=False)

In [7]:
import json

input_file = "/root/bbdd/logs-zeek/iot-23-logs/CTU-IoT-Malware-Capture-36-1/conn_stadistics.log"
output_file = "/root/bbdd/logs-zeek/iot-23-logs/labeled-csv/notpandas-labeled-CTU-IoT-Malware-Capture-36-1.csv"

label_1 = "C&C"
label_2 = "Okiru"
label_3 = "HeartBeat"
label_4 = "Okiru-Attack"

all_keys = ["ts", "startTime", "uid", "id.orig_h", "id.orig_p", "id.resp_h", "id.resp_p", "proto", "service", "duration", "orig_bytes", 
            "resp_bytes", "conn_state", "local_orig", "local_resp", "missed_bytes", "history", "orig_pkts", "orig_ip_bytes", "resp_pkts", 
            "resp_ip_bytes", "tunnel_parents", "orig_bytes_mean", "resp_bytes_mean", "orig_bytes_std", "resp_bytes_std", "orig_bytes_mean_nocero", 
            "resp_bytes_mean_nocero", "orig_bytes_std_nocero", "resp_bytes_std_nocero", "orig_bytes_min", "resp_bytes_min", "orig_bytes_max", 
            "resp_bytes_max", "orig_pkts_nocero", "resp_pkts_nocero", "orig_pkts_cero", "resp_pkts_cero", "time_mean", "time_std", "time_min", 
            "time_max", "orig_time_mean", "orig_time_std", "orig_time_min", "orig_time_max", "resp_time_mean", "resp_time_std", "resp_time_min", "resp_time_max"] 

with open(input_file, "r") as f_in, open(output_file, "w") as f_out:
    f_out.write("ts,startTime,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,orig_bytes_mean,resp_bytes_mean,orig_bytes_std,resp_bytes_std,orig_bytes_mean_nocero,resp_bytes_mean_nocero,orig_bytes_std_nocero,resp_bytes_std_nocero,orig_bytes_min,resp_bytes_min,orig_bytes_max,resp_bytes_max,orig_pkts_nocero,resp_pkts_nocero,orig_pkts_cero,resp_pkts_cero,time_mean,time_std,time_min,time_max,orig_time_mean,orig_time_std,orig_time_min,orig_time_max,resp_time_mean,resp_time_std,resp_time_min,resp_time_max,label,binary-label\n")
    for line in f_in:
        data = json.loads(line)
        for key in all_keys:
            if key not in data:
                data[key] = ""
        binary_label = 0
        labels = []  # Start empty
        # REVISAAAAAAAAAAAAAAAAAAAR
        if data["id.resp_p"] == "57722":
            labels.append(label_1)
            binary_label = 1
        if data["id.resp_p"] == "37215":
            labels.append(label_2)
            binary_label = 1
        if data["id.resp_h"] == "185.130.215.13":
            labels.append(label_3)
            binary_label = 1
        if data["id.resp_h"] == "103.56.26.82":
            labels.append(label_4)
            binary_label = 1
        
        if not labels:
            labels.append("benign")

        if not labels:
            labels.append("benign")
        
        values = [str(data[key]) for key in all_keys]
        labels_joined = '-'.join(labels)
        csv_line = ','.join(values) + f',{labels_joined},{binary_label}\n'        
        f_out.write(csv_line)



## CTU-Malware 39

df['binary-label'] = 0  # Initialize with 0
df['label'] = ''
rule_1 = df['id.resp_p'] == 6667
rule_2 = (df['conn_state'] == 'S0') & (df['id.resp_p'] == 22)
rule_3 = (df['conn_state'] == 'SF') & (df['id.resp_p'] == 22)

df.loc[rule_1, 'label'] = 'C&C'
df.loc[rule_2, 'label'] = "PartOfAHorizontalPortScan"
df.loc[rule_3, 'label'] = "Attack"

## CTU-Malware 42

df['binary-label'] = 0  # Initialize with 0
df['label'] = ''
rule_1 = df['id.resp_h'] == "104.24.97.120"
rule_2 = (df['resp_ip_bytes'] > 50000) 


df.loc[rule_1, 'label'] = 'C&C'
df.loc[rule_2, 'label'] = "FileDownload"

## CTU-Malware 43

rule_1 = df['id.resp_p'] == 45
rule_2 = df['id.resp_h'] == "142.11.219.83"
rule_3 = df['id.resp_p'] == 27015
rule_4 = df['id.resp_p'] == 37215
rule_5 = (df['id.resp_p'] == 52869) & (df['conn_state'] == 'S0')
rule_6 = (df['resp_ip_bytes'] > 50000) 

df.loc[rule_1, 'label'] = "C&C"
df.loc[rule_2, 'label'] = "C&C"
df.loc[rule_3, 'label'] = "DDoS"
df.loc[rule_4, 'label'] = "Okiru"
df.loc[rule_5, 'label'] = "PartOfAHorizontalPortScan"
df.loc[rule_6, 'label'] = "FileDonwload"

In [ ]:
# Define the file path
#file_path = "/root/bbdd/logs-zeek/iot-23-logs/CTU-IoT-Malware-Capture-43-1/conn_stadistics.log"

# Define chunk size
#chunk_size = 10000

# Define the conditions and corresponding labels
#conditions_labels = [
#    (lambda row: row['id.resp_p'] == 45, 'C&C'),
#    (lambda row: row['id.resp_h'] == "142.11.219.83", 'C&C'),
#    (lambda row: row['id.resp_p'] == 27015, 'DDoS'),
#    (lambda row: row['id.resp_p'] == 37215, 'Okiru'),
#    (lambda row: row['id.resp_p'] == 52869 and row['conn_state'] == 'S0', 'PartOfAHorizontalPortScan'),
#    (lambda row: row['resp_ip_bytes'] > 50000, 'FileDownload')
#]

# Process the DataFrame in chunks
#chunks = pd.read_json(file_path, lines=True, chunksize=chunk_size)
#for chunk in chunks:
#    for condition, label in conditions_labels:
#        chunk.loc[condition(chunk), 'label'] = label
    # Save the chunk to a new CSV file
#    chunk.to_csv('/root/bbdd/logs-zeek/iot-23-logs/labeled-csv/labeled-CTU-IoT-Malware-Capture-43-1.csv', mode='a', header=False, index=False)

In [21]:
import json

input_file = "/root/bbdd/logs-zeek/iot-23-logs/CTU-IoT-Malware-Capture-43-1/conn_stadistics.log"
output_file = "/root/bbdd/logs-zeek/iot-23-logs/labeled-csv/json-labeled-CTU-IoT-Malware-Capture-43-1.csv"

label_1 = "C&C"
label_2 = "DDoS"
label_3 = "Okiru"
label_4 = "PartOfAHorizontalPortScan"
label_5 = "FileDonwload"

# Initialize label checks
label_checked = [False] * 5

all_keys = ["ts", "startTime", "uid", "id.orig_h", "id.orig_p", "id.resp_h", "id.resp_p", "proto", "service", "duration", "orig_bytes", 
            "resp_bytes", "conn_state", "local_orig", "local_resp", "missed_bytes", "history", "orig_pkts", "orig_ip_bytes", "resp_pkts", 
            "resp_ip_bytes", "tunnel_parents", "orig_bytes_mean", "resp_bytes_mean", "orig_bytes_std", "resp_bytes_std", "orig_bytes_mean_nocero", 
            "resp_bytes_mean_nocero", "orig_bytes_std_nocero", "resp_bytes_std_nocero", "orig_bytes_min", "resp_bytes_min", "orig_bytes_max", 
            "resp_bytes_max", "orig_pkts_nocero", "resp_pkts_nocero", "orig_pkts_cero", "resp_pkts_cero", "time_mean", "time_std", "time_min", 
            "time_max", "orig_time_mean", "orig_time_std", "orig_time_min", "orig_time_max", "resp_time_mean", "resp_time_std", "resp_time_min", "resp_time_max"]


with open(input_file, "r") as f_in, open(output_file, "w") as f_out:
    f_out.write("ts,startTime,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,orig_bytes_mean,resp_bytes_mean,orig_bytes_std,resp_bytes_std,orig_bytes_mean_nocero,resp_bytes_mean_nocero,orig_bytes_std_nocero,resp_bytes_std_nocero,orig_bytes_min,resp_bytes_min,orig_bytes_max,resp_bytes_max,orig_pkts_nocero,resp_pkts_nocero,orig_pkts_cero,resp_pkts_cero,time_mean,time_std,time_min,time_max,orig_time_mean,orig_time_std,orig_time_min,orig_time_max,resp_time_mean,resp_time_std,resp_time_min,resp_time_max,label,binary-label\n")

    for line in f_in:
        data = json.loads(line)
        for key in all_keys:
            if key not in data:
                data[key] = ""
        binary_label = 0
        labels = []  # Start empty
        if data["id.resp_p"] == 45 and not label_checked[0]:
            labels.append(label_1)
            binary_label = 1
            label_checked[0] = True

        if data["id.resp_h"] == "142.11.219.83" and not label_checked[0]:
            labels.append(label_1)
            binary_label = 1
            label_checked[0] = True

        if data["id.resp_p"] == 27015 and not label_checked[1]:
            labels.append(label_2)
            binary_label = 1
            label_checked[1] = True

        if data["id.resp_p"] == 37215 and not label_checked[2]:
            labels.append(label_3)
            binary_label = 1
            label_checked[2] = True
            
        if data["id.resp_p"] == 52869 and data["conn_state"] == "S0" and not label_checked[3]:
            labels.append(label_4)
            binary_label = 1
            label_checked[3] = True

        if data["resp_ip_bytes"] > 50000 and not label_checked[4]:
            labels.append(label_5)
            binary_label = 1
            label_checked[4] = True
        

        # Reset label checks if all labels are checked
        # If no labels are added, assign "benign"
        if not labels:
            labels.append("benign")
        
        values = [str(data[key]) for key in all_keys]
        labels_joined = '-'.join(labels)
        csv_line = ','.join(values) + f',{labels_joined},{binary_label}\n'        
        f_out.write(csv_line)

        label_checked = [False] * 5

## CTU-Malware 44

rule_1 = (df['id.resp_h'] == "46.101.251.172") & (df['proto'] == "tcp")
rule_2 = (df['id.resp_p'] == 80) & (df['proto'] == "udp")
rule_3 = df['id.resp_h'] == "86.136.151.56" 
rule_4 = (df['resp_ip_bytes'] > 50000) 


df.loc[rule_1, 'label'] = 'C&C'
df.loc[rule_2, 'label'] = 'DDoS'
df.loc[rule_3, 'label'] = 'DDoS'
df.loc[rule_4, 'label'] = 'FileDownload'


In [18]:
import json

input_file = "/root/bbdd/logs-zeek/iot-23-logs/CTU-IoT-Malware-Capture-44-1/conn_stadistics.log"
output_file = "/root/bbdd/logs-zeek/iot-23-logs/labeled-csv/json-labeled-CTU-IoT-Malware-Capture-44-1.csv"

label_1 = "C&C"
label_2 = "DDoS"
label_3 = "FileDonwload"

all_keys = ["ts", "startTime", "uid", "id.orig_h", "id.orig_p", "id.resp_h", "id.resp_p", "proto", "service", "duration", "orig_bytes", 
            "resp_bytes", "conn_state", "local_orig", "local_resp", "missed_bytes", "history", "orig_pkts", "orig_ip_bytes", "resp_pkts", 
            "resp_ip_bytes", "tunnel_parents", "orig_bytes_mean", "resp_bytes_mean", "orig_bytes_std", "resp_bytes_std", "orig_bytes_mean_nocero", 
            "resp_bytes_mean_nocero", "orig_bytes_std_nocero", "resp_bytes_std_nocero", "orig_bytes_min", "resp_bytes_min", "orig_bytes_max", 
            "resp_bytes_max", "orig_pkts_nocero", "resp_pkts_nocero", "orig_pkts_cero", "resp_pkts_cero", "time_mean", "time_std", "time_min", 
            "time_max", "orig_time_mean", "orig_time_std", "orig_time_min", "orig_time_max", "resp_time_mean", "resp_time_std", "resp_time_min", "resp_time_max"]

# Initialize label checks
label_checked = [False] * 3

with open(input_file, "r") as f_in, open(output_file, "w") as f_out:
    #f_out.write("ts,startTime,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,orig_bytes_mean,resp_bytes_mean,orig_bytes_std,resp_bytes_std,orig_bytes_mean_nocero,resp_bytes_mean_nocero,orig_bytes_std_nocero,resp_bytes_std_nocero,orig_bytes_min,resp_bytes_min,orig_bytes_max,resp_bytes_max,orig_pkts_nocero,resp_pkts_nocero,orig_pkts_cero,resp_pkts_cero,time_mean,time_std,time_min,time_max,orig_time_mean,orig_time_std,orig_time_min,orig_time_max,resp_time_mean,resp_time_std,resp_time_min,resp_time_max,service,label,binary-label\n")
    #f_out.write("ts,startTime,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,orig_bytes_mean,resp_bytes_mean,orig_bytes_std,orig_bytes_mean_nocero,resp_bytes_mean_nocero,orig_bytes_std_nocero,resp_bytes_std_nocero,orig_bytes_min,resp_bytes_min,orig_bytes_max,resp_bytes_max,orig_pkts_nocero,resp_pkts_nocero,orig_pkts_cero,resp_pkts_cero,time_mean,time_std,time_min,time_max,orig_time_mean,orig_time_min,orig_time_max,resp_time_mean,resp_time_min,resp_time_max,resp_bytes_std,orig_time_std,resp_time_std,service,label,binary-label\n")
    f_out.write("ts,startTime,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,orig_bytes_mean,resp_bytes_mean,orig_bytes_std,resp_bytes_std,orig_bytes_mean_nocero,resp_bytes_mean_nocero,orig_bytes_std_nocero,resp_bytes_std_nocero,orig_bytes_min,resp_bytes_min,orig_bytes_max,resp_bytes_max,orig_pkts_nocero,resp_pkts_nocero,orig_pkts_cero,resp_pkts_cero,time_mean,time_std,time_min,time_max,orig_time_mean,orig_time_std,orig_time_min,orig_time_max,resp_time_mean,resp_time_std,resp_time_min,resp_time_max,label,binary-label\n")

    for line in f_in:
        data = json.loads(line)
        
        for key in all_keys:
            if key not in data:
                data[key] = ""
                
        binary_label = 0
        labels = []  # Start empty
        if data["id.resp_h"] == "46.101.251.172" and (data['proto'] == "tcp") and not label_checked[0]:
            labels.append(label_1)
            binary_label = 1
            label_checked[0] = True

        if data["id.resp_p"] == 80 and (data['proto'] == "udp") and not label_checked[1]:
            labels.append(label_2)
            binary_label = 1
            label_checked[1] = True

        if data["id.resp_h"] == "86.136.151.56"  and not label_checked[1]:
            labels.append(label_2)
            binary_label = 1
            label_checked[1] = True

        if data['resp_ip_bytes'] > 50000 and not label_checked[2]:
            labels.append(label_3)
            binary_label = 1
            label_checked[2] = True
            
        

        # Reset label checks if all labels are checked
        # If no labels are added, assign "benign"
        if not labels:
            labels.append("benign")
        
        values = [str(data[key]) for key in all_keys]
        labels_joined = '-'.join(labels)
        csv_line = ','.join(values) + f',{labels_joined},{binary_label}\n'        
        f_out.write(csv_line)

        label_checked = [False] * 3

## CTU-Malware 48

rule_1 = (df['id.resp_h'] == "167.99.182.238") 
rule_2 = (df['id.resp_h'] == "167.99.182.238")  & (df['resp_ip_bytes'] > 1) 
rule_3 = (df['id.resp_p'] == 80) & (df['resp_ip_bytes'] > 50000) 
rule_4 = (df['id.resp_p'] == 23) & (df['conn_state'] == 'S0')
rule_5 = (df['id.resp_p'] == 23) & (df['orig_ip_bytes'] > 7) 


df.loc[rule_1, 'label'] = 'C&C'
df.loc[rule_2, 'label'] = 'HeartBeat'
df.loc[rule_3, 'label'] = 'FileDownload'
df.loc[rule_4, 'label'] = 'PartOfAHorizontalPortscan'
df.loc[rule_5, 'label'] = 'Attack'

In [8]:
import json

input_file = "/root/bbdd/logs-zeek/iot-23-logs/CTU-IoT-Malware-Capture-48-1/conn_stadistics.log"
output_file = "/root/bbdd/logs-zeek/iot-23-logs/labeled-csv/json-labeled-CTU-IoT-Malware-Capture-48-1.csv"

label_1 = "C&C"
label_2 = "HeartBeat"
label_3 = "FileDownload"
label_4 = "PartOfAHorizontalPortScan"
label_5 = "Attack"

# Initialize label checks
label_checked = [False] * 5

all_keys = ["ts", "startTime", "uid", "id.orig_h", "id.orig_p", "id.resp_h", "id.resp_p", "proto", "service", "duration", "orig_bytes", 
            "resp_bytes", "conn_state", "local_orig", "local_resp", "missed_bytes", "history", "orig_pkts", "orig_ip_bytes", "resp_pkts", 
            "resp_ip_bytes", "tunnel_parents", "orig_bytes_mean", "resp_bytes_mean", "orig_bytes_std", "resp_bytes_std", "orig_bytes_mean_nocero", 
            "resp_bytes_mean_nocero", "orig_bytes_std_nocero", "resp_bytes_std_nocero", "orig_bytes_min", "resp_bytes_min", "orig_bytes_max", 
            "resp_bytes_max", "orig_pkts_nocero", "resp_pkts_nocero", "orig_pkts_cero", "resp_pkts_cero", "time_mean", "time_std", "time_min", 
            "time_max", "orig_time_mean", "orig_time_std", "orig_time_min", "orig_time_max", "resp_time_mean", "resp_time_std", "resp_time_min", "resp_time_max"]

with open(input_file, "r") as f_in, open(output_file, "w") as f_out:
    f_out.write("ts,startTime,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,orig_bytes_mean,resp_bytes_mean,orig_bytes_std,resp_bytes_std,orig_bytes_mean_nocero,resp_bytes_mean_nocero,orig_bytes_std_nocero,resp_bytes_std_nocero,orig_bytes_min,resp_bytes_min,orig_bytes_max,resp_bytes_max,orig_pkts_nocero,resp_pkts_nocero,orig_pkts_cero,resp_pkts_cero,time_mean,time_std,time_min,time_max,orig_time_mean,orig_time_std,orig_time_min,orig_time_max,resp_time_mean,resp_time_std,resp_time_min,resp_time_max,label,binary-label\n")
    for line in f_in:
        data = json.loads(line)
        for key in all_keys:
            if key not in data:
                data[key] = ""

        binary_label = 0
        labels = []  # Start empty
        if data["id.resp_h"] == "167.99.182.238" and not label_checked[0]:
            labels.append(label_1)
            binary_label = 1
            label_checked[0] = True

        if data["id.resp_h"] == "167.99.182.238" and (data['resp_ip_bytes'] > 1) and not label_checked[1]:
            labels.append(label_2)
            binary_label = 1
            label_checked[1] = True

        if data["id.resp_p"] == 80 and (data['resp_ip_bytes'] > 50000) and not label_checked[2]:
            labels.append(label_3)
            binary_label = 1
            label_checked[2] = True

        if data["id.resp_p"] == 23 and (data['conn_state'] == 'S0') and not label_checked[3]:
            labels.append(label_4)
            binary_label = 1
            label_checked[3] = True
            
        if data["id.resp_p"] == 23 and (data['orig_ip_bytes'] > 7) and not label_checked[4]:
            labels.append(label_5)
            binary_label = 1
            label_checked[4] = True

        

        # Reset label checks if all labels are checked
        # If no labels are added, assign "benign"
        if not labels:
            labels.append("benign")
        
        values = [str(data[key]) for key in all_keys]
        labels_joined = '-'.join(labels)
        csv_line = ','.join(values) + f',{labels_joined},{binary_label}\n'        
        f_out.write(csv_line)

        label_checked = [False] * 5

## CTU-Malware 49

rule_1 = (df['id.resp_p'] == 4554)
rule_2 = (df['id.resp_p'] == 8081) & (df['conn_state'] == 'S0')
rule_3 = (df['resp_ip_bytes'] > 30000) 


df.loc[rule_1, 'label'] = 'C&C'
df.loc[rule_2, 'label'] = 'PartOfAHorizontalPortscan'
df.loc[rule_3, 'label'] = 'FileDownload'

In [11]:
import json

input_file = "/root/bbdd/logs-zeek/iot-23-logs/CTU-IoT-Malware-Capture-49-1/conn_stadistics.log"
output_file = "/root/bbdd/logs-zeek/iot-23-logs/labeled-csv/json-labeled-CTU-IoT-Malware-Capture-49-1.csv"

label_1 = "C&C"
label_2 = "PartOfAHorizontalPortScan"
label_3 = "FileDownload"

# Initialize label checks
label_checked = [False] * 3

all_keys = ["ts", "startTime", "uid", "id.orig_h", "id.orig_p", "id.resp_h", "id.resp_p", "proto", "service", "duration", "orig_bytes", 
            "resp_bytes", "conn_state", "local_orig", "local_resp", "missed_bytes", "history", "orig_pkts", "orig_ip_bytes", "resp_pkts", 
            "resp_ip_bytes", "tunnel_parents", "orig_bytes_mean", "resp_bytes_mean", "orig_bytes_std", "resp_bytes_std", "orig_bytes_mean_nocero", 
            "resp_bytes_mean_nocero", "orig_bytes_std_nocero", "resp_bytes_std_nocero", "orig_bytes_min", "resp_bytes_min", "orig_bytes_max", 
            "resp_bytes_max", "orig_pkts_nocero", "resp_pkts_nocero", "orig_pkts_cero", "resp_pkts_cero", "time_mean", "time_std", "time_min", 
            "time_max", "orig_time_mean", "orig_time_std", "orig_time_min", "orig_time_max", "resp_time_mean", "resp_time_std", "resp_time_min", "resp_time_max"]



with open(input_file, "r") as f_in, open(output_file, "w") as f_out:
    f_out.write("ts,startTime,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,orig_bytes_mean,resp_bytes_mean,orig_bytes_std,resp_bytes_std,orig_bytes_mean_nocero,resp_bytes_mean_nocero,orig_bytes_std_nocero,resp_bytes_std_nocero,orig_bytes_min,resp_bytes_min,orig_bytes_max,resp_bytes_max,orig_pkts_nocero,resp_pkts_nocero,orig_pkts_cero,resp_pkts_cero,time_mean,time_std,time_min,time_max,orig_time_mean,orig_time_std,orig_time_min,orig_time_max,resp_time_mean,resp_time_std,resp_time_min,resp_time_max,label,binary-label\n")
    for line in f_in:
        data = json.loads(line)
        for key in all_keys:
            if key not in data:
                data[key] = ""

        binary_label = 0
        labels = []  # Start empty
        if data["id.resp_p"] == 4554 and not label_checked[0]:
            labels.append(label_1)
            binary_label = 1
            label_checked[0] = True

        if data["id.resp_p"] == 8081 and (data['conn_state'] == 'S0') and not label_checked[1]:
            labels.append(label_2)
            binary_label = 1
            label_checked[1] = True

        if data['resp_ip_bytes'] > 30000 and not label_checked[2]:
            labels.append(label_3)
            binary_label = 1
            label_checked[2] = True

        # Reset label checks if all labels are checked
        # If no labels are added, assign "benign"
        if not labels:
            labels.append("benign")
        
        values = [str(data[key]) for key in all_keys]
        labels_joined = '-'.join(labels)
        csv_line = ','.join(values) + f',{labels_joined},{binary_label}\n'        
        f_out.write(csv_line)

        label_checked = [False] * 3

## CTU-Malware 52

rule_1 = (df['id.resp_h'] == "185.244.25.108") 
rule_1 = (df['id.resp_p'] == 4441)
rule_3 = (df['id.resp_p'] == 80) & (df['resp_ip_bytes'] > 30000) 
rule_4 = (df['id.resp_p'] == 23) 
rule_5 = (df['id.resp_p'] == 2323)


df.loc[rule_1, 'label'] = 'C&C'
df.loc[rule_2, 'label'] = 'Mirai'
df.loc[rule_3, 'label'] = 'FileDownload'
df.loc[rule_4, 'label'] = 'PartOfAHorizontalPortscan'
df.loc[rule_5, 'label'] = 'PartOfAHorizontalPortscan'

In [12]:
import json

input_file = "/root/bbdd/logs-zeek/iot-23-logs/CTU-IoT-Malware-Capture-52-1/conn_stadistics.log"
output_file = "/root/bbdd/logs-zeek/iot-23-logs/labeled-csv/json-labeled-CTU-IoT-Malware-Capture-52-1.csv"

label_1 = "C&C"
label_2 = "Mirai"
label_3 = "FileDownload"
label_4 = "PartOfAHorizontalPortscan"

# Define all possible keys
all_keys = ["ts", "startTime", "uid", "id.orig_h", "id.orig_p", "id.resp_h", "id.resp_p", "proto", "service", "duration", "orig_bytes", 
            "resp_bytes", "conn_state", "local_orig", "local_resp", "missed_bytes", "history", "orig_pkts", "orig_ip_bytes", "resp_pkts", 
            "resp_ip_bytes", "tunnel_parents", "orig_bytes_mean", "resp_bytes_mean", "orig_bytes_std", "resp_bytes_std", "orig_bytes_mean_nocero", 
            "resp_bytes_mean_nocero", "orig_bytes_std_nocero", "resp_bytes_std_nocero", "orig_bytes_min", "resp_bytes_min", "orig_bytes_max", 
            "resp_bytes_max", "orig_pkts_nocero", "resp_pkts_nocero", "orig_pkts_cero", "resp_pkts_cero", "time_mean", "time_std", "time_min", 
            "time_max", "orig_time_mean", "orig_time_std", "orig_time_min", "orig_time_max", "resp_time_mean", "resp_time_std", "resp_time_min", "resp_time_max"]

# Initialize label checks
label_checked = [False] * 4

with open(input_file, "r") as f_in, open(output_file, "w") as f_out:
    f_out.write("ts,startTime,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,orig_bytes_mean,resp_bytes_mean,orig_bytes_std,resp_bytes_std,orig_bytes_mean_nocero,resp_bytes_mean_nocero,orig_bytes_std_nocero,resp_bytes_std_nocero,orig_bytes_min,resp_bytes_min,orig_bytes_max,resp_bytes_max,orig_pkts_nocero,resp_pkts_nocero,orig_pkts_cero,resp_pkts_cero,time_mean,time_std,time_min,time_max,orig_time_mean,orig_time_std,orig_time_min,orig_time_max,resp_time_mean,resp_time_std,resp_time_min,resp_time_max,label,binary-label\n")
    for line in f_in:
        data = json.loads(line)
        for key in all_keys:
            if key not in data:
                data[key] = ""
        binary_label = 0
        labels = []  # Start empty
        if data["id.resp_h"] == "185.244.25.108" and not label_checked[0]:
            labels.append(label_1)
            binary_label = 1
            label_checked[0] = True

        if data["id.resp_p"] == 4441 and not label_checked[1]:
            labels.append(label_2)
            binary_label = 1
            label_checked[1] = True

        if data["id.resp_p"] == 80 and (data['resp_ip_bytes'] > 30000) and not label_checked[2]:
            labels.append(label_3)
            binary_label = 1
            label_checked[2] = True
            
        if data["id.resp_p"] == 23 and not label_checked[3]:
            labels.append(label_4)
            binary_label = 1
            label_checked[3] = True
        
        if data["id.resp_p"] == 2323 and not label_checked[3]:
            labels.append(label_4)
            binary_label = 1
            label_checked[3] = True
        

        # Reset label checks if all labels are checked
        # If no labels are added, assign "benign"
        if not labels:
            labels.append("benign")
        
        values = [str(data[key]) for key in all_keys]
        labels_joined = '-'.join(labels)
        csv_line = ','.join(values) + f',{labels_joined},{binary_label}\n'        
        f_out.write(csv_line)

        label_checked = [False] * 4

## CTU-Malware 60

rule_1 = (df['id.resp_h'] == "102.165.48.81") 
rule_1 = (df['id.resp_p'] == 17769)
rule_3 = (df['id.resp_h'] == "102.165.48.81") 
rule_4 = (df['id.resp_p'] == 52869) 
rule_5 = (df['id.resp_p'] == 80)
rule_6 = (df['id.resp_p'] == 9307)
rule_7 = (df['id.resp_p'] == 20480)
rule_8 = (df['id.resp_p'] == 28416)
rule_9 = (df['id.resp_p'] == 62336)

df.loc[rule_1, 'label'] = 'C&C'
df.loc[rule_2, 'label'] = 'HeartBeat'
df.loc[rule_3, 'label'] = 'DDoS'
df.loc[rule_4, 'label'] = 'DDoS'
df.loc[rule_5, 'label'] = 'DDoS'
df.loc[rule_6, 'label'] = 'DDoS'
df.loc[rule_7, 'label'] = 'DDoS'
df.loc[rule_8, 'label'] = 'DDoS'
df.loc[rule_9, 'label'] = 'DDoS'

## CTU-Malware 7

rule_1 = (df['id.resp_h'] == "185.130.215.13") 
rule_2 = (df['id.resp_h'] == "102.157.125.155") 
rule_3 = (df['id.resp_p'] == 37215) 
rule_4 = (df['id.resp_p'] == 57722)
rule_5 = (df['id.resp_p'] == 80)


df.loc[rule_1, 'label'] = 'C&C'
df.loc[rule_2, 'label'] = 'Okiru'
df.loc[rule_3, 'label'] = 'Okiru'
df.loc[rule_4, 'label'] = 'HeartBeat'
df.loc[rule_5, 'label'] = 'DDoS'

In [ ]:
import json

input_file = "/root/bbdd/logs-zeek/iot-23-logs/CTU-IoT-Malware-Capture-7-1/conn_stadistics.log"
output_file = "/root/bbdd/logs-zeek/iot-23-logs/labeled-csv/json-labeled-CTU-IoT-Malware-Capture-7-1.csv"

label_1 = "C&C"
label_2 = "Okiru"
label_3 = "HeartBeat"
label_4 = "DDoS"

# Define all possible keys
all_keys = ["ts", "startTime", "uid", "id.orig_h", "id.orig_p", "id.resp_h", "id.resp_p", "proto", "service", "duration", "orig_bytes", 
            "resp_bytes", "conn_state", "local_orig", "local_resp", "missed_bytes", "history", "orig_pkts", "orig_ip_bytes", "resp_pkts", 
            "resp_ip_bytes", "tunnel_parents", "orig_bytes_mean", "resp_bytes_mean", "orig_bytes_std", "resp_bytes_std", "orig_bytes_mean_nocero", 
            "resp_bytes_mean_nocero", "orig_bytes_std_nocero", "resp_bytes_std_nocero", "orig_bytes_min", "resp_bytes_min", "orig_bytes_max", 
            "resp_bytes_max", "orig_pkts_nocero", "resp_pkts_nocero", "orig_pkts_cero", "resp_pkts_cero", "time_mean", "time_std", "time_min", 
            "time_max", "orig_time_mean", "orig_time_std", "orig_time_min", "orig_time_max", "resp_time_mean", "resp_time_std", "resp_time_min", "resp_time_max"]

# Initialize label checks
label_checked = [False] * 4

with open(input_file, "r") as f_in, open(output_file, "w") as f_out:
    f_out.write("ts,startTime,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,orig_bytes_mean,resp_bytes_mean,orig_bytes_std,resp_bytes_std,orig_bytes_mean_nocero,resp_bytes_mean_nocero,orig_bytes_std_nocero,resp_bytes_std_nocero,orig_bytes_min,resp_bytes_min,orig_bytes_max,resp_bytes_max,orig_pkts_nocero,resp_pkts_nocero,orig_pkts_cero,resp_pkts_cero,time_mean,time_std,time_min,time_max,orig_time_mean,orig_time_std,orig_time_min,orig_time_max,resp_time_mean,resp_time_std,resp_time_min,resp_time_max,label,binary-label\n")
    for line in f_in:
        data = json.loads(line)
        for key in all_keys:
            if key not in data:
                data[key] = ""
        binary_label = 0
        labels = []  # Start empty
        if data["id.resp_h"] == "185.130.215.13" and not label_checked[0]:
            labels.append(label_1)
            binary_label = 1
            label_checked[0] = True

        if data["id.resp_h"] == "102.157.125.155" and not label_checked[1]:
            labels.append(label_2)
            binary_label = 1
            label_checked[1] = True

        if data["id.resp_p"] == 37215 and not label_checked[1]:
            labels.append(label_2)
            binary_label = 1
            label_checked[1] = True
            
        if data["id.resp_p"] == 57722 and not label_checked[2]:
            labels.append(label_3)
            binary_label = 1
            label_checked[2] = True
        
        if data["id.resp_p"] == 80 and not label_checked[3]:
            labels.append(label_4)
            binary_label = 1
            label_checked[3] = True
        

        # Reset label checks if all labels are checked
        # If no labels are added, assign "benign"
        if not labels:
            labels.append("benign")
        
        values = [str(data[key]) for key in all_keys]
        labels_joined = '-'.join(labels)
        csv_line = ','.join(values) + f',{labels_joined},{binary_label}\n'        
        f_out.write(csv_line)

        label_checked = [False] * 4

## CTU-Malware 8

rule_1 = (df['id.resp_p'] == 50)

df.loc[rule_1, 'label'] = 'C&C'

## CTU-Malware 9

rule_1 = (df['id.resp_p'] == 81) & (df['conn_state'] == 'S0')
rule_2 = (df['id.resp_p'] == 23) & (df['conn_state'] == 'S0')

df.loc[rule_1, 'label'] = 'PartOfAHorizontalPortscan'
df.loc[rule_2, 'label'] = 'PartOfAHorizontalPortscan'

# Etiquetado HoneyPot Benign

In [5]:
input_file = "/root/bbdd/logs-zeek/iot-23-logs/CTU-Honeypot-Capture-4-1/conn_stadistics.log"
output_file = "/root/bbdd/logs-zeek/iot-23-logs/labeled-csv/json-labeled-CTU-IoT-CTU-IoT-Honeypot-Capture-4-1.csv"

In [7]:
import pandas as pd

# Read the JSON file into a DataFrame
df = pd.read_json(input_file, lines=True)

# Create two new columns with default values
df['label'] = 'benign'
df['binary-label'] = 0
df.to_csv(output_file, index=False)


# Display the DataFrame
df.head()

,ts,startTime,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,conn_state,...,resp_time_max,duration,orig_bytes,resp_bytes,orig_bytes_std,orig_time_std,resp_bytes_std,resp_time_std,label,binary-label
0,1.536925e+09,2018-09-14 11:40:31,CWX5XALyzLn7sN7kg,fe80::5bcc:698e:39d5:cdf,5353,ff02::fb,5353,udp,dns,S0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,benign,0
1,1.536925e+09,2018-09-14 11:40:43,CpRpSU15jYqJc1ZAQa,fe80::66db:8bff:fe21:4ccf,5353,ff02::fb,5353,udp,dns,S0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,benign,0
2,1.536925e+09,2018-09-14 11:40:46,CnVNqa3lOpG3CAbM9b,fe80::5bcc:698e:39d5:cdf,5353,ff02::fb,5353,udp,dns,S0,...,0.0,15.02909,1089.0,0.0,60.637522,2.348012,NaN,NaN,benign,0
3,1.536925e+09,2018-09-14 11:41:17,CVq7xh12YJUO1HciCb,fe80::5bcc:698e:39d5:cdf,5353,ff02::fb,5353,udp,dns,S0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,benign,0
4,1.536925e+09,2018-09-14 11:40:44,CFm39E2Y3RRBmLLjV7,::,135,ff02::1:ffd5:cdf,136,icmp,NaN,OTH,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,benign,0


In [8]:
input_file = "/root/bbdd/logs-zeek/iot-23-logs/CTU-Honeypot-Capture-5-1/conn_stadistics.log"
output_file = "/root/bbdd/logs-zeek/iot-23-logs/labeled-csv/json-labeled-CTU-IoT-CTU-IoT-Honeypot-Capture-5-1.csv"

In [9]:
import pandas as pd

# Read the JSON file into a DataFrame
df = pd.read_json(input_file, lines=True)

# Create two new columns with default values
df['label'] = 'benign'
df['binary-label'] = 0
df.to_csv(output_file, index=False)


# Display the DataFrame
df.head()

,ts,startTime,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,...,orig_time_std,orig_time_min,orig_time_max,resp_time_mean,resp_time_min,resp_time_max,resp_bytes_std,resp_time_std,label,binary-label
0,1.537523e+09,2018-09-21 09:40:22,Cd6Od74GyKiPhFWaHl,0.0.0.0,68,255.255.255.255,67,udp,dhcp,8.322388,...,0.00000,8.322388,8.322388,0.0,10000.0,0.0,NaN,NaN,benign,0
1,1.537523e+09,2018-09-21 09:41:37,CPlv8R2vatG8nPtuZ4,192.168.2.1,5353,224.0.0.251,5353,udp,dns,NaN,...,NaN,10000.000000,0.000000,0.0,10000.0,0.0,NaN,NaN,benign,0
2,1.537523e+09,2018-09-21 09:41:37,Ci6pms3OO8vTbHQ0s9,fe80::80e6:50ff:fe12:1464,5353,ff02::fb,5353,udp,dns,NaN,...,NaN,10000.000000,0.000000,0.0,10000.0,0.0,NaN,NaN,benign,0
3,1.537523e+09,2018-09-21 09:41:37,CLi1Ne3wvwQiKt3AFd,169.254.15.115,5353,224.0.0.251,5353,udp,dns,NaN,...,NaN,10000.000000,0.000000,0.0,10000.0,0.0,NaN,NaN,benign,0
4,1.537523e+09,2018-09-21 09:41:39,Ctcw7H2SBhJ7MmcrHk,0.0.0.0,68,255.255.255.255,67,udp,dhcp,60.555587,...,2.66683,1.933225,8.899929,0.0,10000.0,0.0,NaN,NaN,benign,0


In [10]:
input_file = "/root/bbdd/logs-zeek/iot-23-logs/CTU-Honeypot-Capture-7-1/Somfy-01/conn_stadistics.log"
output_file = "/root/bbdd/logs-zeek/iot-23-logs/labeled-csv/json-labeled-CTU-IoT-CTU-IoT-Honeypot-Capture-7-1.csv"

In [11]:
import pandas as pd

# Read the JSON file into a DataFrame
df = pd.read_json(input_file, lines=True)

# Create two new columns with default values
df['label'] = 'benign'
df['binary-label'] = 0
df.to_csv(output_file, index=False)


# Display the DataFrame
df.head()

,ts,startTime,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,...,orig_time_std,orig_time_min,orig_time_max,resp_time_mean,resp_time_min,resp_time_max,resp_bytes_std,resp_time_std,label,binary-label
0,1.562160e+09,2019-07-03 13:17:29,C2pmwl2FK9arA5z4Bi,192.168.1.1,67,255.255.255.255,68,udp,dhcp,0.004564,...,0.001216,0.000012,0.004412,0.0,10000.0,0.0,NaN,NaN,benign,0
1,1.562160e+09,2019-07-03 13:16:59,Ce40A03hv92hSrB6fd,0.0.0.0,68,255.255.255.255,67,udp,dhcp,30.004642,...,3.010847,0.000013,11.999909,0.0,10000.0,0.0,NaN,NaN,benign,0
2,1.562160e+09,2019-07-03 13:19:13,CQg9Bb3hk4MWg4Wiy7,fe80::5bcc:698e:39d5:cdf,5353,ff02::fb,5353,udp,dns,3.948539,...,0.750759,0.200711,2.123718,0.0,10000.0,0.0,NaN,NaN,benign,0
3,1.562160e+09,2019-07-03 13:19:58,CzzBSr5zmS3iEcQl6,fe80::5bcc:698e:39d5:cdf,5353,ff02::fb,5353,udp,dns,3.768179,...,0.710945,0.200722,2.033478,0.0,10000.0,0.0,NaN,NaN,benign,0
4,1.562160e+09,2019-07-03 13:20:24,CogGeM2ZajS8Xwndzh,fe80::4eef:c0ff:fe27:561e,5353,ff02::fb,5353,udp,dns,0.000114,...,0.000005,0.000009,0.000027,0.0,10000.0,0.0,NaN,NaN,benign,0
